In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from azureml.core import Model
from azureml.core import Experiment
from azureml.core import Workspace

In [2]:
import azureml.core
print(f'AZURE ML SDK VERSION: {azureml.core.VERSION}')

AZURE ML SDK VERSION: 1.30.0


In [3]:
ws = Workspace.from_config()
print(ws)

Workspace.create(name='mlops_ws', subscription_id='6f507867-3280-4f79-8820-9939bef45222', resource_group='learn_mlops')


### Define Environnement

In [4]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

In [5]:
env = Environment(name='my_env')

In [6]:
for pip_package in ["numpy", "onnxruntime", "joblib", "azureml-core", "azureml-monitoring", "azureml-defaults", 
                    "scikit-learn==0.22.2", "inference-schema", "inference-schema[numpy-support]"]:
    env.python.conda_dependencies.add_pip_package(pip_package)
    
inference_config = InferenceConfig(entry_script='score.py', environment=env)

In [7]:
env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210513.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "my_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
       

### Deployment configuration

In [8]:
from azureml.core.webservice import AksWebservice

deploy_config = AksWebservice.deploy_configuration(collect_model_data=True)
deploy_config

### Deploy webservice

In [9]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [12]:
# name for cluster Kubernetes
ask_name = 'myask-cluster'
#ask_target = ComputeTarget(workspace=ws, name=ask_name)
#print('Found existing cluster Kubernetes....')
# provisioning configuration with default parameters
prov_config = AksCompute.provisioning_configuration()
ask_target = ComputeTarget.create(workspace = ws, name = ask_name, provisioning_configuration = prov_config)
ask_target.wait_for_completion(show_output = True)

Creating..
FailedProvisioning operation finished, operation "Failed"


ComputeTargetException: ComputeTargetException:
	Message: Compute object provisioning polling reached non-successful terminal state, current provisioning state: Failed
Provisioning operation error:
{'code': 'InvalidTemplateDeployment', 'message': "The template deployment '18adf9a1-3717-47f1-9239-593e6253df61' is not valid according to the validation procedure. The tracking id is '584c4672-323a-419c-b55f-a208d0950829'. See inner errors for details.", 'details': [{'code': 'QuotaExceeded', 'message': 'Provisioning of resource(s) for container service myask-cluster1fe57155b in resource group Learn_MLOps failed. Message: {\n  "code": "QuotaExceeded",\n  "message": "Provisioning of resource(s) for container service myask-cluster1fe57155b in resource group Learn_MLOps failed. Message: Operation could not be completed as it results in exceeding approved Total Regional Cores quota. Additional details - Deployment Model: Resource Manager, Location: northeurope, Current Limit: 4, Current Usage: 0

In [ ]:
model1 = Model(workspace=ws, name='Scaler')
model2 = Model(workspace=ws, name="Support-vector-Classifier")